# Project Introduction: Integrating RSI and Bollinger Bands for Stock Analysis

## Objective
The objective of this project is to develop a reliable stock screening tool that identifies potential buy and sell signals using two popular technical indicators: the Relative Strength Index (RSI) and Bollinger Bands. By combining these indicators, we aim to enhance the accuracy of our signals, leveraging the strengths of both momentum and volatility analysis.

## Background
Technical analysis involves using historical price and volume data to forecast future price movements. Among the myriad of technical indicators available, RSI and Bollinger Bands are two widely respected tools that traders use to assess market conditions.

## Why RSI and Bollinger Bands?

### Relative Strength Index (RSI)
- **Purpose:** RSI is a momentum oscillator that measures the speed and change of price movements. It oscillates between 0 and 100.
- **Overbought/Oversold Conditions:** RSI values above 70 typically indicate that a stock is overbought, suggesting it might be overvalued and due for a pullback. Conversely, RSI values below 30 indicate that a stock is oversold, suggesting it might be undervalued and due for a rise.
- **Chosen Thresholds:** We chose 70 and 30 as our thresholds because they are standard in technical analysis, providing a balanced view of overbought and oversold conditions without being too conservative or too aggressive.

### Bollinger Bands
- **Purpose:** Bollinger Bands measure market volatility and provide relative high and low price levels. They consist of a middle band (a simple moving average) and two outer bands (standard deviations above and below the middle band).
- **Price Extremes:** When the price moves above the upper Bollinger Band, it indicates that the asset is relatively overbought, similar to the RSI's indication. When the price moves below the lower Bollinger Band, it indicates that the asset is relatively oversold.
- **Complementary Nature:** Bollinger Bands add a volatility perspective to RSI's momentum perspective. While RSI alone might show overbought conditions, combining it with Bollinger Bands helps to confirm whether the price is also at an extreme high relative to its recent volatility.

## Combining RSI and Bollinger Bands
By integrating RSI and Bollinger Bands, we enhance our ability to identify potential market turning points. Here’s how these indicators complement each other:
- **Enhanced Accuracy:** While RSI helps identify potential overbought and oversold conditions based on momentum, Bollinger Bands provide context by showing whether these conditions occur at price extremes relative to recent volatility.
- **Filter for False Signals:** RSI can sometimes give premature signals. By requiring that the price also be outside the Bollinger Bands, we filter out some of these false signals, increasing the likelihood that our signals are accurate.

## Methodology

- **Data Collection:** We collect historical price data for a selection of stocks over a specified period.
- **Indicator Calculation:** We calculate the RSI and Bollinger Bands for each stock.
- **Stock Screening:** We screen stocks based on the following criteria:
  - **Sell Criteria:** RSI > 70 (overbought) and price above the upper Bollinger Band.
  - **Buy Criteria:** RSI < 30 (oversold) and price below the lower Bollinger Band.
- **Visualization and Export:** We visualize the results and export the screened stocks to a CSV file for further analysis.

## Data Source and Flexibility

In this analysis, we will be using data from the FTSE 250 companies. However, feel free to use any dataset you prefer by simply changing the CSV file name in the script. Ensure that your dataset contains a column named "Ticker".

## Conclusion

This project aims to leverage the complementary strengths of RSI and Bollinger Bands to create an intriguing stock screening tool. By considering both momentum and volatility, our signals offer interesting insights designed to inspire further analysis. The chosen thresholds are standard and widely accepted in technical analysis, providing a thought-provoking starting point for potential future exploration.


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import contextlib
import sys
import os
from tqdm import tqdm

# Function to suppress print statements
@contextlib.contextmanager
def suppress_output():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

# Function to calculate Relative Strength Index (RSI)
def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))
    return data

# Function to calculate Bollinger Bands
def calculate_bollinger_bands(data, window=20, num_std_dev=2):
    data['MA20'] = data['Close'].rolling(window=window).mean()
    data['BB_Upper'] = data['MA20'] + (data['Close'].rolling(window=window).std() * num_std_dev)
    data['BB_Lower'] = data['MA20'] - (data['Close'].rolling(window=window).std() * num_std_dev)
    return data

# Define threshold parameters
rsi_overbought = 70  # RSI threshold to determine overbought condition
rsi_oversold = 30    # RSI threshold to determine oversold condition

# Function to analyze stock for RSI and Bollinger Bands
def analyze_stock_rsi_bb(symbol):
    try:
        # Fetch historical price data for the last 7 months
        end_date = datetime.now() - timedelta(days=1)
        start_date = end_date - timedelta(days=9*30)  # Approx. 7 months

        # Suppress yfinance output
        with suppress_output():
            data = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
        
        if data.empty:
            print(f"No data fetched for {symbol}")
            return None

        # Calculate RSI and Bollinger Bands
        data = calculate_rsi(data)
        data = calculate_bollinger_bands(data)

        data.dropna(subset=['RSI', 'MA20', 'BB_Upper', 'BB_Lower'], inplace=True)

        if data.empty:
            print(f"Insufficient data after calculating indicators for {symbol}")
            return None

        # Determine if the stock meets any of the criteria
        latest_rsi = data['RSI'].iloc[-1]
        latest_close = data['Close'].iloc[-1]
        bb_upper = data['BB_Upper'].iloc[-1]
        bb_lower = data['BB_Lower'].iloc[-1]
        is_overbought = latest_rsi > rsi_overbought
        is_oversold = latest_rsi < rsi_oversold
        price_above_bb_upper = latest_close > bb_upper
        price_below_bb_lower = latest_close < bb_lower

        # Determine if the stock meets both criteria
        meets_criteria = (is_overbought and price_above_bb_upper) or (is_oversold and price_below_bb_lower)

        return {
            "Symbol": symbol,
            "Data": data,
            "RSI": latest_rsi,
            "Close": latest_close,
            "BB_Upper": bb_upper,
            "BB_Lower": bb_lower,
            "Overbought": is_overbought,
            "Oversold": is_oversold,
            "Price_Above_BB_Upper": price_above_bb_upper,
            "Price_Below_BB_Lower": price_below_bb_lower,
            "Meets Criteria": meets_criteria
        }

    except Exception as e:
        print(f"Error analyzing stock {symbol}: {e}")
        return None

# Load tickers from the CSV file
tickers_df = pd.read_csv('ftse_250_tickers.csv')
symbols = tickers_df['Ticker'].tolist()

# Initialize list to store results
results = []

# Analyze each stock
for symbol in tqdm(symbols, desc="Processing stocks"):
    stock_data = analyze_stock_rsi_bb(symbol)
    if stock_data is not None and stock_data["Meets Criteria"]:  # Only include stocks meeting the criteria
        results.append(stock_data)

# Convert results to DataFrame
results_df = pd.DataFrame([{
    "Symbol": r["Symbol"],
    "RSI": r["RSI"],
    "Close": r["Close"],
    "BB_Upper": r["BB_Upper"],
    "BB_Lower": r["BB_Lower"],
    "Overbought": r["Overbought"],
    "Oversold": r["Oversold"],
    "Price_Above_BB_Upper": r["Price_Above_BB_Upper"],
    "Price_Below_BB_Lower": r["Price_Below_BB_Lower"],
    "Meets Criteria": r["Meets Criteria"]
} for r in results])

# Export results to CSV
results_df.to_csv("rsi_bb_screener_results.csv", index=False)

# Print results
results_df


In [ ]:
# Calculate and print the detailed results
total_tickers = len(tickers_df)
tickers_meeting_criteria = len(results_df)
percentage_meeting_criteria = (tickers_meeting_criteria / total_tickers) * 100

# Print results
print("My preference is a 5-20% threshold which strikes a balance, ensuring a\nfocused yet diverse selection of stocks to allow further investigations.")
print(f"Tickers meeting criteria: {tickers_meeting_criteria}")
print(f"Percentage of shares that pass the criteria: {percentage_meeting_criteria:.2f}%")

# Determine if the percentage is within the desired filter range
if 5 <= percentage_meeting_criteria <= 20:
    print("The percentage of shares meeting the criteria is within the desirable range (5-20%).")
else:
    print("The percentage of shares meeting the criteria is outside the desirable range (5-15%).")

In [ ]:
# Function to print basic financial data
def print_financial_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info

    def format_value(value, currency=False):
        if isinstance(value, (int, float)):
            formatted_value = f"{value:,}"
            if currency:
                formatted_value = f"£{formatted_value}"
            return formatted_value
        return value

    market_cap = format_value(info.get('marketCap', 'N/A'), currency=True)
    shares_outstanding = format_value(info.get('sharesOutstanding', 'N/A'))
    year_high = info.get('fiftyTwoWeekHigh', 'N/A')
    year_low = info.get('fiftyTwoWeekLow', 'N/A')
    pe_ratio = info.get('trailingPE', 'N/A')
    eps = info.get('trailingEps', 'N/A')
    dividend_yield = info.get('dividendYield', 'N/A')
    payout_ratio = info.get('payoutRatio', 'N/A')
    revenue = format_value(info.get('totalRevenue', 'N/A'), currency=True)
    gross_profit = format_value(info.get('grossProfits', 'N/A'), currency=True)
    net_income = format_value(info.get('netIncomeToCommon', 'N/A'), currency=True)
    total_debt = format_value(info.get('totalDebt', 'N/A'), currency=True)
    operating_cash_flow = format_value(info.get('operatingCashflow', 'N/A'), currency=True)
    free_cash_flow = format_value(info.get('freeCashflow', 'N/A'), currency=True)
    sector = info.get('sector', 'N/A')
    industry = info.get('industry', 'N/A')
    description = info.get('longBusinessSummary', 'N/A')

    print(f"{'Ticker:':<25} {ticker}")
    print(f"{'Company:':<25} {info.get('shortName', 'N/A')}")
    print(f"{'Sector:':<25} {sector}")
    print(f"{'Industry:':<25} {industry}")
    print(f"{'Market Cap:':<25} {market_cap}")
    print(f"{'Shares Outstanding:':<25} {shares_outstanding}")
    print(f"{'52-Week High:':<25} {year_high}")
    print(f"{'52-Week Low:':<25} {year_low}")
    print(f"{'PE Ratio (TTM):':<25} {pe_ratio}")
    print(f"{'EPS (TTM):':<25} {eps}")
    print(f"{'Dividend Yield:':<25} {dividend_yield}")
    print(f"{'Payout Ratio:':<25} {payout_ratio}")
    print(f"{'Revenue (TTM):':<25} {revenue}")
    print(f"{'Gross Profit (TTM):':<25} {gross_profit}")
    print(f"{'Net Income (TTM):':<25} {net_income}")
    print(f"{'Total Debt:':<25} {total_debt}")
    print(f"{'Operating Cash Flow (TTM):':<25} {operating_cash_flow}")
    print(f"{'Free Cash Flow (TTM):':<25} {free_cash_flow}")
    print(f"\n{'Description:':<25} {description}")  # Print the full description
    print("-" * 50)




# Function to plot RSI and Bollinger Bands indicators
def plot_rsi_bb(data, symbol, rsi_overbought, rsi_oversold):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

    # Plot Close Price and Bollinger Bands
    ax1.plot(data.index, data['Close'], label='Close Price', color='blue')
    ax1.plot(data.index, data['BB_Upper'], label='Upper Bollinger Band', color='red')
    ax1.plot(data.index, data['BB_Lower'], label='Lower Bollinger Band', color='green')
    ax1.set_title(f'{symbol} - Close Price and Bollinger Bands')
    ax1.set_ylabel('Close Price')
    ax1.legend(loc='upper left')
    ax1.grid(True)

    # Plot RSI
    ax2.plot(data.index, data['RSI'], label='RSI', color='purple')
    ax2.axhline(y=rsi_overbought, color='red', linestyle='--', linewidth=2, label='Overbought Threshold (70)')
    ax2.axhline(y=rsi_oversold, color='green', linestyle='--', linewidth=2, label='Oversold Threshold (30)')
    ax2.set_title('RSI')
    ax2.set_ylabel('RSI Value')
    ax2.legend(loc='upper left')
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

# Print the introductory text
print("The following stocks have met the specified criteria. A plot is displayed showing the stock price and its relationship to the associated indicators. Additionally, detailed financial and company information is provided for each stock.\n")

# Iterate through all tickers to plot graphs and print financial data
for stock_data in results: 
    symbol = stock_data["Symbol"]
    data = stock_data["Data"]
    plot_rsi_bb(data, symbol, rsi_overbought, rsi_oversold)  # Call plot_rsi_bb function with stock data and symbol
    print_financial_data(symbol)  # Call print_financial_data function with symbol

